In [1]:
import datetime as dt
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import cartopy.feature
from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs
import xarray as xr
import math
import netCDF4
from pathlib import Path
import requests

In [2]:
ds_urlg = 'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/202102/e5.oper.an.pl.128_129_z.ll025sc.2021021400_2021021423.nc'
ds_urlt = 'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/202102/e5.oper.an.pl.128_130_t.ll025sc.2021021400_2021021423.nc'

In [3]:
HOME = str(Path.home())
credFile = open(HOME+'/.rdarc','r')
userId, pw = credFile.read().split()

In [4]:
## first, lets set some info about the times, level, etc., we want to plot

lev2plot = 500           #


year = 2021          
month = 2
day = 14
total_days = 1           # how many days are we plotting?
first_fhr = 0            # setting to 0 = 00 UTC, 6 = 06 UTC, ...
hours = total_days * 24  # total hours in time range to plot
hr_inc = 6               # the delta time for the data

date1 = dt.datetime(year,month,day,0)  # first date to plot

#create an array of times we want to plot...
times = [date1 + dt.timedelta(hours=x) for x in range(0,hours,hr_inc)]
#...and put them in the correct time units "hours since 00 UTC 1 Jan 1800 "
date_list = netCDF4.date2num(times,units="hours since 1900-01-01 00:00:00",calendar="gregorian") #change dates to netcdf times


In [5]:
# Lets open our data file and get out data

# climo data is in /erai/climo/mean/VAR.nc and /erai/climo/stdev/VAR.nc
session = requests.Session()
session.auth = (userId, pw)

store_g = xr.backends.PydapDataStore.open(ds_urlg, session=session)
store_t = xr.backends.PydapDataStore.open(ds_urlt, session=session)

gfile = xr.open_dataset(store_g)  ##Change to your data/file
g_var = gfile["Z"]
g_data  = (g_var.loc[dict(level=lev2plot,latitude=slice(90.,0) )]) / 9.81 ## div by gravity to get geopot. height

tfile = xr.open_dataset(store_t)
t_var = tfile["T"]
t_data  = (t_var.loc[dict(level=lev2plot,latitude=slice(90.,0) )]) 
print(g_data.coords)
lev = g_data["level"].values
lat = g_data["latitude"].values
lon = g_data["longitude"].values



Coordinates:
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... 0.75 0.5 0.25 0.0
    level      float64 500.0
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time       (time) datetime64[ns] 2021-02-14 ... 2021-02-14T23:00:00


In [6]:
for date in date_list: #loop through each day 
    ind = np.where(date_list == date)[0]
    for d in ind:
        height = g_data.loc[dict(time=times[d] )]
        temperature = t_data.loc[dict(time=times[d] )]
        print(height.coords)
        
        print("Date "+times[d].strftime("%H UTC %d %b %Y"))
        valid_label = (times[d].strftime("Valid: %H UTC %d %b %Y"))

        plt.Figure(figsize=(15,15),dpi=120)    ## <---Set fig size here!

        #Set plot as an orthographic projection looking down at the Earth from the pole with 0˚ at 6-oclock
        ax = plt.axes(projection=ccrs.Orthographic(0,90))     
        ax.add_feature(cartopy.feature.LAND, zorder=1, edgecolor='dimgray',facecolor='none')
        ax.outline_patch.set_edgecolor('none')
        gl = ax.gridlines(color="grey",linestyle=":",linewidth=0.5)  
        glevs = range(1000,8000,100)  ## <---set contour interval for geo. heights
        tlevs = range(0,400,4)        ## <--- set contour interval for temp
        [x,y] = np.meshgrid(lon,lat)  ## <--- set lat/lon as coords on grid
        
        plt.title(str(lev2plot)+"hPa Geopotential Height & Temp \n"+valid_label)
        hghtplot = ax.contour(x,y,height,glevs,transform=ccrs.PlateCarree(),extend='both',colors='black',linewidth=1.0)
        ax.clabel(hghtplot, inline=True, fmt="%.0f", fontsize=8)
        tempplot = ax.contourf(x,y,temperature,100,transform=ccrs.PlateCarree(),cmap="cool")
        plt.colorbar(tempplot)

        plot_label = times[d].strftime("%Y%m%d%H")
        plt.savefig(f"{plot_label}_{lev2plot}hpa.png",format='png')
        plt.clf()
        

Coordinates:
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... 0.75 0.5 0.25 0.0
    level      float64 500.0
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
    time       datetime64[ns] 2021-02-14
Date 00 UTC 14 Feb 2021


/kt11/ktyle/anaconda3_2001/envs/jun20/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: The outline_patch property is deprecated. Use GeoAxes.spines['geo'] or the default Axes properties instead.
  app.launch_new_instance()
/kt11/ktyle/anaconda3_2001/envs/jun20/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1478: UserWarning: The following kwargs were not used by contour: 'linewidth'
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


Coordinates:
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... 0.75 0.5 0.25 0.0
    level      float64 500.0
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
    time       datetime64[ns] 2021-02-14T06:00:00
Date 06 UTC 14 Feb 2021
Coordinates:
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... 0.75 0.5 0.25 0.0
    level      float64 500.0
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
    time       datetime64[ns] 2021-02-14T12:00:00
Date 12 UTC 14 Feb 2021
Coordinates:
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... 0.75 0.5 0.25 0.0
    level      float64 500.0
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
    time       datetime64[ns] 2021-02-14T18:00:00
Date 18 UTC 14 Feb 2021
